### 自己动手编写一个3层全连接分类网络

In [10]:
import torch
import torch.nn as nn
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import numpy as np
import time
import datetime

### 载入了 CIFAR10 10分类数据集

In [2]:
dataset = datasets.CIFAR10(r'C:\Users\Lin_hz\data',train= True,transform=transforms.Compose([
                            transforms.ToTensor(),
                            transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))

]))

In [3]:
train_data = torch.utils.data.DataLoader(dataset,batch_size=4,shuffle=True)

In [4]:
class MLP(nn.Module):
    def __init__(self,img_size):
        super(MLP,self).__init__()
        self.img_size = img_size
        self.model = nn.Sequential(
        nn.Linear(3*self.img_size*self.img_size,1024),
        nn.ReLU(inplace=True),
        nn.BatchNorm1d(1024),
        nn.Linear(1024,100),
        nn.ReLU(inplace=True),
        nn.BatchNorm1d(100),
        nn.Linear(100,10),
        )
    
    def forward(self,x):
        output = x.view(-1,3*self.img_size*self.img_size)
        output = self.model(output)
        
        return output

In [5]:
def weight_init(m):
    layer_name = m.__class__.__name__
    if layer_name.find("Linear") != -1:
        nn.init.normal_(m.weight.data)

In [6]:
device = torch.device('cuda' if torch.cuda.is_available else 'cpu')
# 模型
model = MLP(dataset[0][0].shape[1]).to(device)

# 初始化
model.apply(weight_init)

MLP(
  (model): Sequential(
    (0): Linear(in_features=3072, out_features=1024, bias=True)
    (1): ReLU(inplace)
    (2): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Linear(in_features=1024, out_features=100, bias=True)
    (4): ReLU(inplace)
    (5): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): Linear(in_features=100, out_features=10, bias=True)
  )
)

In [7]:
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(),lr=0.01,momentum=0.9,weight_decay=1e-5)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer,[10,],gamma=0.1)

In [11]:
for epoch_i in range(30):
    scheduler.step()
    start_time = time.time()
    for i,(img,label) in enumerate(train_data):
        img = img.requires_grad_(True).to(device)
        label = torch.tensor(label).to(device)
        output = model(img)
        loss = loss_func(output,label)
        
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        time_left = datetime.timedelta(seconds=(len(train_data)-(i+1)) * (time.time() - start_time) / (i + 1))
        
        if (i+1) % 2500 == 0 :
            print(f'Train Epoch: {epoch_i} [{i}/{len(train_data)} Loss: {loss.data}]')
            print(f'ETA {time_left}'  )

C:\Users\Lin_hz\Anaconda3\envs\env_new\lib\site-packages\ipykernel_launcher.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


Train Epoch: 0 [0/12500 Loss: 5.5566911697387695]
ETA 0:05:36.563482
Train Epoch: 0 [2500/12500 Loss: 3.798368453979492]
ETA 0:01:09.870055
Train Epoch: 0 [5000/12500 Loss: 1.9861782789230347]
ETA 0:00:52.466608
Train Epoch: 0 [7500/12500 Loss: 1.9616122245788574]
ETA 0:00:34.962119
Train Epoch: 0 [10000/12500 Loss: 1.2703269720077515]
ETA 0:00:17.460036
Train Epoch: 1 [0/12500 Loss: 1.1804698705673218]
ETA 0:01:52.509667
Train Epoch: 1 [2500/12500 Loss: 2.259579658508301]
ETA 0:01:09.299875
Train Epoch: 1 [5000/12500 Loss: 2.3636789321899414]
ETA 0:00:52.007491
Train Epoch: 1 [7500/12500 Loss: 1.5589385032653809]
ETA 0:00:34.742779
Train Epoch: 1 [10000/12500 Loss: 2.263279914855957]
ETA 0:00:17.376302
Train Epoch: 2 [0/12500 Loss: 2.1833763122558594]
ETA 0:01:52.172927
Train Epoch: 2 [2500/12500 Loss: 2.215794801712036]
ETA 0:01:09.363666
Train Epoch: 2 [5000/12500 Loss: 1.770560622215271]
ETA 0:00:52.152555
Train Epoch: 2 [7500/12500 Loss: 1.5536837577819824]
ETA 0:00:34.734139
Trai

Train Epoch: 22 [10000/12500 Loss: 2.305758476257324]
ETA 0:00:17.067533
Train Epoch: 23 [0/12500 Loss: 1.877901554107666]
ETA 0:02:17.124416
Train Epoch: 23 [2500/12500 Loss: 1.5707371234893799]
ETA 0:01:08.538289
Train Epoch: 23 [5000/12500 Loss: 1.5081915855407715]
ETA 0:00:51.506500
Train Epoch: 23 [7500/12500 Loss: 1.793548345565796]
ETA 0:00:34.244945
Train Epoch: 23 [10000/12500 Loss: 2.269786834716797]
ETA 0:00:17.109400
Train Epoch: 24 [0/12500 Loss: 1.9803205728530884]
ETA 0:01:52.077568
Train Epoch: 24 [2500/12500 Loss: 2.4282217025756836]
ETA 0:01:07.968066
Train Epoch: 24 [5000/12500 Loss: 2.2217674255371094]
ETA 0:00:51.202897
Train Epoch: 24 [7500/12500 Loss: 1.2475064992904663]
ETA 0:00:34.127293
Train Epoch: 24 [10000/12500 Loss: 1.9897000789642334]
ETA 0:00:17.053077
Train Epoch: 25 [0/12500 Loss: 2.5246143341064453]
ETA 0:01:52.184847
Train Epoch: 25 [2500/12500 Loss: 2.2933335304260254]
ETA 0:01:07.848490
Train Epoch: 25 [5000/12500 Loss: 2.156374931335449]
ETA 0:00

In [12]:
output

tensor([[-1.4690, -2.3412,  0.8872,  0.6097,  0.8977,  0.8558,  1.8368,  0.5649,
         -1.5599, -1.3189],
        [ 2.9101, -0.6749,  0.3035, -0.7280, -0.4876, -1.0610, -1.9720, -0.7218,
          1.1142, -0.9825],
        [ 0.2765,  0.8876, -0.5191,  0.0599, -0.6984, -0.4116, -2.3738,  0.2197,
         -0.4269,  1.4753],
        [-1.4414,  1.9443, -0.1751,  0.3604,  0.4662,  0.4390,  1.7379,  0.0072,
          0.1391,  0.6935]], device='cuda:0', grad_fn=<AddmmBackward>)

In [13]:
import torch.nn.functional as F

In [29]:
pre = F.softmax(output,dim=1).argmax(dim=1)

In [39]:
torch.eq(pre,label).sum().float().item() / 4.0

0.75

In [41]:
test_set = datasets.CIFAR10(r'C:\Users\Lin_hz\data',train= False,transform=transforms.Compose([
                            transforms.ToTensor(),
                            transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))]))
test_data = torch.utils.data.DataLoader(dataset,batch_size=4,shuffle=False)

In [68]:
losses = 0
accuracys = 0
for i,(img,label) in enumerate(test_data):
        img = img.requires_grad_(True).to(device)
        label = torch.tensor(label).to(device)
        output = model(img)
        loss = loss_func(output,label)
        losses += loss
        
        pred = F.softmax(output,dim=1).argmax(dim=1)
        accu = torch.eq(pred,label).sum().float().item() / len(label)
        accuracys += accu
        if (i+1) % 2500 == 0:
            print(f'Test [{i+1}/{len(test_data)}] Loss: {losses.item()/ (i+1):2.4f}\
 Accuracy: {accuracys/(i+1)*100 : 2.2f}%')

C:\Users\Lin_hz\Anaconda3\envs\env_new\lib\site-packages\ipykernel_launcher.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """


Test [2500/12500] Loss: 1.8524 Accuracy:  34.08%
Test [5000/12500] Loss: 1.8613 Accuracy:  33.92%
Test [7500/12500] Loss: 1.8588 Accuracy:  34.01%
Test [10000/12500] Loss: 1.8602 Accuracy:  34.08%
Test [12500/12500] Loss: 1.8640 Accuracy:  33.95%


In [69]:
torch.__version__

'1.1.0'